In [88]:
import requests
import pandas as pd
from secret_configs import api_key
import jinja2 as j2


In [89]:
# From Exchange Rates API Doc https://exchangeratesapi.io/documentation/

"""
request data with the following parameters:
- base: AUD
- start date: 01/01/2022
- end date: 31/12/2022
"""
# base_currency = "AUD"
# start_day = "2012-05-01"
# end_day = "2012-05-25"
# base_url = f"https://api.exchangeratesapi.io/v1/timeseries?access_key={api_key}&start_date={start_day}&end_date={end_day}&base={base_currency}"

# response_data = []

# params = {
#    "start_date" : start_day,
#    "end_date" : end_day,
#    "base" : base_currency,
#    "access_key" : api_key
#}

start_date = "2022-01-01"
end_date = "2022-12-31"
symbols = "USD,SGD,JPY,EUR,CNY,INR"
base = "AUD"

url = f"https://api.apilayer.com/exchangerates_data/timeseries?start_date={start_date}&end_date={end_date}&symbols={symbols}&base={base}"

payload = {}
headers= {
  "apikey": "TyfX4zH87A9CmeZI7CUWHpCayinLIXKF"
}

response = requests.request("GET", url, headers=headers, data = payload)

status_code = response.status_code
result = response.json()

In [90]:
# check the response
print(status_code)

200


In [91]:
print(result)

{'success': True, 'timeseries': True, 'start_date': '2022-01-01', 'end_date': '2022-12-31', 'base': 'AUD', 'rates': {'2022-01-01': {'USD': 0.72685, 'SGD': 0.980266, 'JPY': 83.666257, 'EUR': 0.639188, 'CNY': 4.619788, 'INR': 54.160524}, '2022-01-02': {'USD': 0.72665, 'SGD': 0.979822, 'JPY': 83.666832, 'EUR': 0.638932, 'CNY': 4.618517, 'INR': 54.145628}, '2022-01-03': {'USD': 0.719682, 'SGD': 0.973872, 'JPY': 83.007413, 'EUR': 0.636897, 'CNY': 4.574371, 'INR': 53.54634}, '2022-01-04': {'USD': 0.724058, 'SGD': 0.981641, 'JPY': 84.108709, 'EUR': 0.6417, 'CNY': 4.601607, 'INR': 53.967765}, '2022-01-05': {'USD': 0.72215, 'SGD': 0.980538, 'JPY': 83.858587, 'EUR': 0.63831, 'CNY': 4.589996, 'INR': 53.749877}, '2022-01-06': {'USD': 0.716342, 'SGD': 0.974868, 'JPY': 82.995693, 'EUR': 0.634104, 'CNY': 4.572408, 'INR': 53.309819}, '2022-01-07': {'USD': 0.718055, 'SGD': 0.974889, 'JPY': 83.01938, 'EUR': 0.631996, 'CNY': 4.579542, 'INR': 53.471788}, '2022-01-08': {'USD': 0.7179, 'SGD': 0.973099, 'JPY

In [92]:
type(result)

dict

In [93]:
# showing the data

df_currency = pd.DataFrame.from_dict(result)
df_currency.reset_index(inplace=True)
df_currency

,index,success,timeseries,start_date,end_date,base,rates
0,2022-01-01,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.72685, 'SGD': 0.980266, 'JPY': 83.66..."
1,2022-01-02,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.72665, 'SGD': 0.979822, 'JPY': 83.66..."
2,2022-01-03,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.719682, 'SGD': 0.973872, 'JPY': 83.0..."
3,2022-01-04,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.724058, 'SGD': 0.981641, 'JPY': 84.1..."
4,2022-01-05,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.72215, 'SGD': 0.980538, 'JPY': 83.85..."
...,...,...,...,...,...,...,...
360,2022-12-27,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.672789, 'SGD': 0.906984, 'JPY': 89.8..."
361,2022-12-28,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.674698, 'SGD': 0.910393, 'JPY': 90.4..."
362,2022-12-29,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.6765, 'SGD': 0.909023, 'JPY': 89.937..."
363,2022-12-30,True,True,2022-01-01,2022-12-31,AUD,"{'USD': 0.6805, 'SGD': 0.912281, 'JPY': 89.223..."


In [94]:
# Doing Transformation in SQL
# Skipping transformation step
# Loading data straight to SQL

from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float, JSON
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from secret_configs import db_user, db_password, db_server_name, db_database_name
from sqlalchemy.schema import CreateTable

In [95]:
# create connection to database

connection_url = URL.create(
    drivername = "postgresql+pg8000",
    username = db_user,
    password = db_password,
    host = db_server_name,
    port = 5432,
    database = db_database_name,
)

engine = create_engine(connection_url)


In [96]:
meta = MetaData()
sql_aud_conversion_2022_table = Table(
    "raw_sql_version_aud_conversion_2022", meta,
    Column("index", String, primary_key=True),
    Column("success", String),
    Column("timeseries", String),
    Column("start_date", String),
    Column("end_date", String),
    Column("base", String),
    Column("rates", JSON),
)
meta.create_all(engine)

In [97]:
# UPSERT STATEMENT

insert_statement = postgresql.insert(sql_aud_conversion_2022_table).values(df_currency.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['index'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['index']})
engine.execute(upsert_statement)

In [98]:
# Using Jinja

target_table = "stg_currency"
with open(f"models/{target_table}.sql") as f:
    raw_sql = f.read()

print(raw_sql)

drop table if exists {{target_table}};

create table {{target_table}} as (
	with cte_currency as
		(select
			CAST(index as DATE) as individual_date,
			CONCAT(TO_CHAR(CAST(index as DATE), 'MONTH'), ' - ', DATE_PART('year', CAST(index as DATE))) as year_month,
			base,
			cast(rates ->> 'USD' as numeric) as usd,  
			cast(rates ->> 'SGD' as numeric) as sgd, 
			cast(rates ->> 'JPY' as numeric) as jpy, 
			cast(rates ->> 'EUR' as numeric) as eur, 
			cast(rates ->> 'CNY' as numeric) as cny, 
			cast(rates ->> 'INR' as numeric) as inr
		from public.raw_sql_version_aud_conversion_2022
		)
	select 	cte_currency.year_month,
	cte_currency.base,
	ROUND(avg(cte_currency.usd), 3) as usd,
	ROUND(avg(cte_currency.sgd), 3) as sgd,
	ROUND(avg(cte_currency.jpy), 3) as jpy,
	ROUND(avg(cte_currency.eur), 3) as eur,
	ROUND(avg(cte_currency.cny), 3) as cny,
	ROUND(avg(cte_currency.inr), 3) as inr
	from cte_currency
	group by cte_currency.year_month, cte_currency.base
)   


In [99]:
parsed_sql = j2.Template(raw_sql).render(target_table = "stg_currency")
print(parsed_sql)
# execute parsed sql
engine.execute(parsed_sql)

drop table if exists stg_currency;

create table stg_currency as (
	with cte_currency as
		(select
			CAST(index as DATE) as individual_date,
			CONCAT(TO_CHAR(CAST(index as DATE), 'MONTH'), ' - ', DATE_PART('year', CAST(index as DATE))) as year_month,
			base,
			cast(rates ->> 'USD' as numeric) as usd,  
			cast(rates ->> 'SGD' as numeric) as sgd, 
			cast(rates ->> 'JPY' as numeric) as jpy, 
			cast(rates ->> 'EUR' as numeric) as eur, 
			cast(rates ->> 'CNY' as numeric) as cny, 
			cast(rates ->> 'INR' as numeric) as inr
		from public.raw_sql_version_aud_conversion_2022
		)
	select 	cte_currency.year_month,
	cte_currency.base,
	ROUND(avg(cte_currency.usd), 3) as usd,
	ROUND(avg(cte_currency.sgd), 3) as sgd,
	ROUND(avg(cte_currency.jpy), 3) as jpy,
	ROUND(avg(cte_currency.eur), 3) as eur,
	ROUND(avg(cte_currency.cny), 3) as cny,
	ROUND(avg(cte_currency.inr), 3) as inr
	from cte_currency
	group by cte_currency.year_month, cte_currency.base
)   


In [100]:
# TRANSFORM DATA USING SQL

#staging_table = "stg_currency"

#engine.execute(f"drop table if exists {staging_table}")

#engine.execute("""
    #create table stg_currency as (
	#select
		#CAST(index as DATE) as individual_date,
		#CONCAT(TO_CHAR(CAST(index as DATE), 'MONTH'), ' - ', DATE_PART('year', CAST(index as DATE))) as year_month,
		#base,
		#cast(rates ->> 'USD' as float) as usd,  
		#cast(rates ->> 'SGD' as float) as sgd, 
		#cast(rates ->> 'JPY' as float) as jpy, 
		#cast(rates ->> 'EUR' as float) as eur, 
		#cast(rates ->> 'CNY' as float) as cny, 
		#cast(rates ->> 'INR' as float) as inr
	#from public.raw_sql_version_aud_conversion_2022
#)   
#""")